In [7]:
%pip install pandas openpyxl transformers torch
%pip install llama-cpp-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
from llama_cpp import Llama
from concurrent.futures import ThreadPoolExecutor

model_path = "/Users/solar/OneDrive/Documents/Capstone/Capstone-Jupyter/mistral-7b-instruct-v0.1.Q4_K_M.gguf"
max_context = 4096
llm = Llama(
    model_path,
    n_ctx=max_context
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/solar/OneDrive/Documents/Capstone/Capstone-Jupyter/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 12

In [9]:
import json
import re

with open("metadata.json", "r", encoding="utf-8") as f:
    raw_metadata = json.load(f)

# Create a lookup table
metadata_lookup = {
    entry["code"]: f'{entry["name"]}. {entry["description"]}' 
    for entry in raw_metadata
}

def extract_codes(query):
    return re.findall(r"\[([A-Za-z0-9_]+)\]", query)

# Find the entry with the longest name + description combo
longest_entry = max(metadata_lookup.items(), key=lambda item: len(item[1]))

# Print the result
print(f'Longest entry code: {longest_entry[0]}')
print(f'Length: {len(longest_entry[1])}')
print(f'Content: {longest_entry[1]}')


Longest entry code: EUTaxManIntSerElcRevOverAlign
Length: 1353
Content: EU Taxon - Man, Instal, Serv of Elec Equip Overall Align. EU Taxon - Man, Instal, Serv of Elec Equip Overall Align: This factor identifies the overall alignment for Manufacture, installation, and servicing of high, medium and low voltage electrical equipment for electrical transmission and distribution that result in or enable a substantial contribution to climate change mitigation, covering the substantial contribution criteria, the do no significant harm criteria, and the minimum social safeguards. This is the aggregated result across all Taxonomy objectives. The possible values are: Aligned,Aligned (>90%),Aligned (>80%),Aligned (>70%),Aligned (>60%),Aligned (>50%),Aligned (>40%),Aligned (>30%),Aligned (>20%),Aligned (>10%),Aligned (>0%),Likely aligned (100%),Likely aligned (>90%),Likely aligned (>80%),Likely aligned (>70%),Likely aligned (>60%),Likely aligned (>50%),Likely aligned (>40%),Likely aligned (>30%),Li

In [10]:
"""
def translate_query(query):
    prompt = f"### Instruction:\nRephrase this ESGish query as a natural English sentence. Each query is asking for all companies or issuers that match some paramater.\n\n{query}\n\n### Response:"

    # Use llama_cpp tokenizer to get exact number of tokens in prompt
    prompt_tokens = llm.tokenize(prompt.encode("utf-8"))
    num_prompt_tokens = len(prompt_tokens)

    max_total_tokens = 512
    max_response_tokens = max_total_tokens - num_prompt_tokens

    if max_response_tokens < 1:
        print(f"Skipping query — prompt too long: {query}")
        return "[Prompt too long]"

    response = llm(prompt, max_tokens=max_response_tokens, temperature=0.1)
    return response["choices"][0]["text"].strip()
"""
"""
def translate_query(query):
    codes = extract_codes(query)
    context = "\n".join(
        f"{code}: {metadata_lookup.get(code, 'No metadata found.')}"
        for code in codes
    )
    
    prompt = f"### Instruction:
Rephrase the following ESGish query into a concise natural English sentence. Each query is asking for all companies or issuers that match some paramater. Use the following metadata definitions for clarity:

{context}

Query: {query}

### Response:"

    response = llm(prompt, max_tokens=2048, temperature=0.1)
    return response["choices"][0]["text"].strip()
"""
def max_tokens(text, max_tokens=40):
    tokens = llm.tokenize(text.encode("utf-8"))
    if len(tokens) <= max_tokens:
        return text
    truncated = llm.detokenize(tokens[:max_tokens]).decode("utf-8", errors="ignore")
    return truncated + "..."

def max_afforded_tokens(codes):
    count = len(codes)
    m_tokens = int(max_context/count)
    if m_tokens < 100:
        m_tokens = 100
    return m_tokens


def translate_query(query, max_total_tokens=2048, max_output_tokens=256):
    codes = extract_codes(query)

    m_tokens = max_afforded_tokens(codes)

    # Build full context blocks for each code
    context_blocks = [
        f"{code}: {max_tokens(metadata_lookup.get(code, 'No metadata found.'), max_tokens=m_tokens)}" for code in codes
    ]

    # Initial prompt pieces
    instruction = "### Instruction:\nRephrase the following ESGish query into a concise natural English sentence. Each query is asking for all companies or issuers that match some paramater. Use the following metadata definitions for clarity:\n\n"
    query_part = f"\n\nQuery: {query}\n\n### Response:"

    # Tokenize instruction and query to calculate remaining token budget
    tokenizer = llm.tokenize  # Built-in tokenizer
    instruction_tokens = len(tokenizer(instruction.encode("utf-8")))
    query_tokens = len(tokenizer(query_part.encode("utf-8")))
    token_budget = max_total_tokens - max_output_tokens - instruction_tokens - query_tokens

    # Now iteratively add context blocks until budget is exhausted
    context = ""
    used_tokens = 0
    for block in context_blocks:
        block_tokens = len(tokenizer(block.encode("utf-8"))) + 1  # +1 for newline
        if used_tokens + block_tokens <= token_budget:
            context += block + "\n"
            used_tokens += block_tokens
        else:
            break  # stop once we're out of budget

    # Final prompt
    prompt = instruction + context + query_part

    # Call model
    response = llm(prompt, max_tokens=max_output_tokens, temperature=0.1)
    return response["choices"][0]["text"].strip()



In [ ]:
import pandas as pd
from functools import partial
import time

#Reads and stores the Esgish queries
df = pd.read_excel("esgish_short.xlsx")
queries = df["Esgish"].tolist()
#Ensures no overload and efficiency
batch_size = 100 
output_file = "translated_queries_TEST.xlsx"

#Looks at each query in each batch, calls the translate_query function, and stores it
for i in range(0, len(queries), batch_size):
    batch = queries[i:i + batch_size]
    translated_batch = []
    
    for query in batch:
        translated = translate_query(query)
        translated_batch.append(translated)
    
    df_batch = pd.DataFrame({
        "Esgish": batch,
        "English": translated_batch
    })

    #Makes a new file if needed, or adds onto the current file during each batch in case the program crashes at some point
    if i == 0:
        df_batch.to_excel(output_file, index=False)  
    else:
        with pd.ExcelWriter(output_file, mode="a", engine="openpyxl", if_sheet_exists="overlay") as writer:
            df_batch.to_excel(writer, index=False, header=False, startrow=i + 1)
    
    print(f"Saved batch {i // batch_size + 1} to {output_file}")

llama_perf_context_print:        load time =    4309.42 ms
llama_perf_context_print: prompt eval time =    4309.21 ms /   175 tokens (   24.62 ms per token,    40.61 tokens per second)
llama_perf_context_print:        eval time =    6558.97 ms /    44 runs   (  149.07 ms per token,     6.71 tokens per second)
llama_perf_context_print:       total time =   10876.28 ms /   219 tokens
Llama.generate: 164 prefix-match hit, remaining 12 prompt tokens to eval
llama_perf_context_print:        load time =    4309.42 ms
llama_perf_context_print: prompt eval time =     300.51 ms /    12 tokens (   25.04 ms per token,    39.93 tokens per second)
llama_perf_context_print:        eval time =    6279.60 ms /    43 runs   (  146.04 ms per token,     6.85 tokens per second)
llama_perf_context_print:       total time =    6587.40 ms /    55 tokens
Llama.generate: 60 prefix-match hit, remaining 116 prompt tokens to eval
llama_perf_context_print:        load time =    4309.42 ms
llama_perf_context_print:

Saved batch 1 to translated_queries_TEST.xlsx


llama_perf_context_print:        load time =    4309.42 ms
llama_perf_context_print: prompt eval time =    3373.68 ms /   132 tokens (   25.56 ms per token,    39.13 tokens per second)
llama_perf_context_print:        eval time =    4794.73 ms /    31 runs   (  154.67 ms per token,     6.47 tokens per second)
llama_perf_context_print:       total time =    8173.88 ms /   163 tokens
Llama.generate: 50 prefix-match hit, remaining 88 prompt tokens to eval
llama_perf_context_print:        load time =    4309.42 ms
llama_perf_context_print: prompt eval time =    2595.72 ms /    88 tokens (   29.50 ms per token,    33.90 tokens per second)
llama_perf_context_print:        eval time =    6621.84 ms /    41 runs   (  161.51 ms per token,     6.19 tokens per second)
llama_perf_context_print:       total time =    9225.41 ms /   129 tokens
Llama.generate: 107 prefix-match hit, remaining 47 prompt tokens to eval
llama_perf_context_print:        load time =    4309.42 ms
llama_perf_context_print: 

Saved batch 2 to translated_queries_TEST.xlsx


llama_perf_context_print:        load time =    4309.42 ms
llama_perf_context_print: prompt eval time =    6476.40 ms /   268 tokens (   24.17 ms per token,    41.38 tokens per second)
llama_perf_context_print:        eval time =    6724.66 ms /    43 runs   (  156.39 ms per token,     6.39 tokens per second)
llama_perf_context_print:       total time =   13209.04 ms /   311 tokens
Llama.generate: 145 prefix-match hit, remaining 203 prompt tokens to eval
llama_perf_context_print:        load time =    4309.42 ms
llama_perf_context_print: prompt eval time =    4795.85 ms /   203 tokens (   23.62 ms per token,    42.33 tokens per second)
llama_perf_context_print:        eval time =   12684.75 ms /    82 runs   (  154.69 ms per token,     6.46 tokens per second)
llama_perf_context_print:       total time =   17496.53 ms /   285 tokens
Llama.generate: 50 prefix-match hit, remaining 1668 prompt tokens to eval
llama_perf_context_print:        load time =    4309.42 ms
llama_perf_context_prin

KeyboardInterrupt: 